In [21]:
open System.Net.Http
open System.Text.Json
open System.Text.Json.Serialization
open System
open System.IO
open System.Net.Http.Headers

In [ ]:
type Credentials() =
    member val username: string = null with get, set 
    member val key: string = null with get,set
      
    static member LoadFrom(path: string): Credentials =
        use reader = new StreamReader(path)
        let json = reader.ReadToEnd()
        JsonSerializer.Deserialize<Credentials>(json)

In [ ]:
let creds = Credentials.LoadFrom("kaggle.json")
creds

In [ ]:
let BaseApiUrl = "https://www.kaggle.com/api/v1/"

type AuthorizedClient = AuthorizedClient of HttpClient

In [ ]:
let CreateAuthorizedClient(auth: Credentials) =
    let authToken =
        sprintf "%s:%s" auth.username auth.key
        |> Text.ASCIIEncoding.ASCII.GetBytes
        |> Convert.ToBase64String

    let client = new HttpClient()
    client.DefaultRequestHeaders.Authorization <- AuthenticationHeaderValue("Basic", authToken)

    AuthorizedClient client

let DownLoadFileAsync (urlPath: string[]) destinationFolder (AuthorizedClient client) =
    let url = sprintf "%sdatasets/download/%s" BaseApiUrl <| String.Join("/", urlPath)
    let filename = urlPath |> Array.last

    async {
        use! stream = client.GetStreamAsync(url) |> Async.AwaitTask
        use fstream = new FileStream(Path.Combine(destinationFolder, filename), FileMode.CreateNew)
        let! _ = stream.CopyToAsync fstream |> Async.AwaitTask
        fstream.Close()
        stream.Close()
    }

In [ ]:
let authClient = CreateAuthorizedClient creds
let (AuthorizedClient client) = authClient

In [ ]:
DownLoadFileAsync [|"selfishgene";"historical-hourly-weather-data.zip"|] "./" authClient
|> Async.RunSynchronously


In [255]:
let DownloadFileAsync 
        (url: string) (destinationFile: string) cancellationToken 
        (report: int64 * float -> unit) (client: HttpClient) =
        async {
            let bufferLength = 8192
            use! response = client.GetAsync(url, HttpCompletionOption.ResponseHeadersRead, cancellationToken)
                            |> Async.AwaitTask
            
            response.EnsureSuccessStatusCode() |> ignore

            let total =
                if response.Content.Headers.ContentLength.HasValue
                then response.Content.Headers.ContentLength.Value
                else -1L
                |> float

            use! contentStream = response.Content.ReadAsStreamAsync() |> Async.AwaitTask
            use fileStream =
                new FileStream(destinationFile, FileMode.Create, FileAccess.Write, FileShare.None, bufferLength, true)

            let mutable totalRead = 0L
            let mutable totalReads = 0L
            let mutable isMoreToRead = true
            let buffer = Array.create bufferLength 0uy

            while isMoreToRead && not cancellationToken.IsCancellationRequested do
                let! read = contentStream.ReadAsync(buffer, 0, bufferLength) |> Async.AwaitTask
                if read > 0 then
                    do! fileStream.WriteAsync(buffer, 0, read) |> Async.AwaitTask
                    totalRead <- totalRead + int64 read
                    totalReads <- totalReads + 1L
                else
                    isMoreToRead <- false

                report (totalRead, float totalRead / total)
        }

In [256]:
open System
open Microsoft.DotNet.Interactive.FSharp
open Microsoft.DotNet.Interactive
open Microsoft.AspNetCore.Html 
open FSharpKernelHelpers 

let progLog (bytesRead : int64,percentage: float) =     
    updateProgressBar "the-bar" (percentage*100.0)    

progressBar "the-bar" |> display

25%

In [258]:
DownloadFileAsync 
    "https://allhdwallpapers.com/wp-content/uploads/2015/06/The-Witcher-3-game-2.png" "test.png" 
    (CancellationToken()) progLog (HttpClient())
|> Async.RunSynchronously
|> ignore

img.["src","test.png"] |> display

In [252]:
let progressBar className =     
    sprintf "<div class='%s progress'>
                 <div class='progress-bar' role='progressbar'
                      style='width: 25%%;' aria-valuenow='25' aria-valuemin='0'
                      aria-valuemax='100'>25%%</div>
             </div>" className
    |> HtmlString 

let updateProgressBar className value =
    sprintf "$('.%s.progress .progress-bar')
                .css('width','%.02f%%')
                .prop('aria-valuenow',%.02f)
                .text('%.02f%%');" className value value value
    |> Javascript
    |> ignore

//updateProgressBar "ref" 25.5

In [ ]:
open FSharpPocketViewTags 

let inline (=>) (key:string) value = key,box value

type Style([<ParamArray>]props: (string*obj)[]) =
    let _props = props
    override _.ToString() =
        let result = Array.fold (fun state (key,value) -> sprintf "%s; %s: %O" state key value) null _props
        result.Trim([|';';' '|])
    member x.ToTuple() =
        "style" => x.ToString()

let prkogressBar className = 
    div.["class",className].
        ["style","border: solid orange 1px; border-radius: 5px;padding: 0"].
        innerHTML (
            span.["class","pbar"].
                 ["style","background-color:lime;text-align: right; margin: -1px;"].
                 innerHTML
                    (span.["class","label-text"].innerHTML("0.0 %") )
        )
        
div.["id","Scar"].
    ["style",Style("border"=>"solid red 1px","padding"=>"10px")].
    innerHTML
        [div.["style",Style("border"=>"solid green 3px","padding"=>"10px")]
         div.["style", "padding:10px;background-color:blue"]]
//|> display

Javascript("$('#Scar>div:last-child').css('background-color','brown').text(jQuery.fn.jquery);")

In [232]:
updateProgressBar "pbar" 45.5

In [32]:
//DownloadFileAsync "https://i.redd.it/u5pz59lmxocy.png" "./jupyter-test.png" (CancellationToken()) progLog (HttpClient())
//|> Async.RunSynchronously

In [24]:
open System
open System.Net.Http
open System.Threading.Tasks
open System.Threading

In [99]:
let k ([<ParamArray>]x) = Array.iter(printfn "%A") x

In [167]:
div.["id","Scar"].
    [Style("border"=>"solid red 1px","padding"=>"10px").ToTuple()]

Unhandled Exception: input.fsx (1,1)-(2,67) typecheck error An indexer property must be given at least one argument

; padding: 5px; border: solid red 1px; font-size: 15.231213

In [130]:
type Style([<ParamArray>]props: (string*obj)[]) =
    let _props = props
    override _.ToString() =
        let result = Array.fold (fun state (key,value) -> sprintf "%s; %s: %O" state key value) null _props
        result.Trim([|';';' '|])
        
Style("padding"=>"5px", "border"=>"solid red 1px", "font-size"=>15.231213)        

padding: 5px; border: solid red 1px; font-size: 15.231213

In [148]:
Javascript("alert('eara')")